Final workflow data processing - VALIDATED:
- add fasta format if missing
- create a column with only aa to remove duplicates
- create column with fasta format to create full fasta
- create new column with duplicated identifiers so i dont miss info
- remove duplicates lines based on sequence
- create fasta file in fasta format

In [ ]:
import pandas as pd
from Bio import Entrez, SeqIO
import time

Entrez.email = "catiarosario10@gmail.com"
excel_path = "RBP_validated_final.xlsx"
sheet_name = "Sheet1"
phage_id_col = "Bacteriophage"
protein_id_col = "RBP Accession"
desc_col = "Protein Type"
source_col = "Bacteriophage"
seq_col = "RBP sequence (aa)"
output_fasta_file = "validated_RBP.fasta"
output_excel_file = "validated_processed.xlsx"

def clean_sequence(seq):
    if not isinstance(seq, str):
        return ""
    lines = seq.strip().splitlines()
    lines = [line for line in lines if not line.startswith(">")]
    return "".join(lines).replace(" ", "").upper()

def make_fasta_header(row):
    desc = row.get(desc_col) if pd.notna(row.get(desc_col)) else ""
    source = row.get(source_col) if pd.notna(row.get(source_col)) else ""
    return f">{row[protein_id_col]} {desc} [{source}]".strip()

def make_full_fasta(row):
    header = make_fasta_header(row)
    seq = row["CleanSeq"]
    return f"{header}\n{seq}"

def get_taxid_and_accession(phage_name):
    try:
        search = Entrez.esearch(db="nucleotide", term=phage_name, retmode="xml")
        record = Entrez.read(search)
        search.close()
        if not record["IdList"]:
            return None, None
        accession_id = record["IdList"][0]
        handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="gb", retmode="text")
        seq_record = SeqIO.read(handle, "genbank")
        handle.close()
        taxid = None
        for feature in seq_record.features:
            if feature.type == "source":
                for ref in feature.qualifiers.get("db_xref", []):
                    if ref.startswith("taxon:"):
                        taxid = ref.split(":")[1]
                        break
        return taxid, accession_id
    except Exception as e:
        print(f"[{phage_name}] TaxID/Accession fetch error: {e}")
        return None, None

df = pd.read_excel(excel_path, sheet_name=sheet_name)
df = df.dropna(subset=[phage_id_col, protein_id_col, seq_col])

df["CleanSeq"] = df[seq_col].apply(clean_sequence)

df["FASTA"] = df.apply(make_full_fasta, axis=1)

seq_to_phageids = {}
for _, row in df.iterrows():
    seq = row["CleanSeq"]
    phageid = row[phage_id_col]
    seq_to_phageids.setdefault(seq, []).append(phageid)

def get_duplicate_phageids(row):
    seq = row["CleanSeq"]
    phageid = row[phage_id_col]
    duplicates = [pid for pid in seq_to_phageids.get(seq, []) if pid != phageid]
    return ", ".join(duplicates) if duplicates else ""

df["DuplicatePhageIDs"] = df.apply(get_duplicate_phageids, axis=1)

df_unique = df.drop_duplicates(subset=["CleanSeq"]).copy()

required_cols = ["Bacteriophage", "Phage accession nr", "Protein Type", "Host", "RBP Accession", "Morphotype"]
missing_cols = [col for col in required_cols if col not in df_unique.columns]
if missing_cols:
    raise ValueError(f"Missing columns: {missing_cols}")

df_unique["AccessionFetched"] = df_unique["Phage accession nr"]
df_unique["Protein Type"] = df_unique["Protein Type"]
df_unique["Bacteriophage"] = df_unique["Bacteriophage"]  

with open(output_fasta_file, "w", encoding="utf-8") as fasta_out:
    for fasta_entry in df_unique["FASTA"]:
        fasta_out.write(fasta_entry + "\n")

cols_to_save = ["Bacteriophage", "Host", "Protein Type", "RBP Accession", "Phage accession nr", "Morphotype", "CleanSeq", "FASTA"]

df_unique.to_excel(output_excel_file, columns=cols_to_save, index=False)
print(f"FASTA saved to '{output_fasta_file}'")
print(f"Excel saved to '{output_excel_file}'")


Final workflow data processing - VALIDATED AND NON-VALIDATED:
- fetch the id of bacteriophages from name
- add column with ids
- add fasta format if missing
- create a column with only aa to remove duplicates
- create column with fasta format to create full fasta
- create new column with duplicated identifiers so i dont miss info
- remove duplicates lines based on sequence
- retrieve morphotypes
- create fasta file in fasta format

In [ ]:
import pandas as pd
from Bio import Entrez, SeqIO
import time

# === Configuration ===
Entrez.email = "catiarosario10@gmail.com"
excel_path = "data_test_code.xlsx"
sheet_name = "data_test"
phage_id_col = "Bacteriophage"
protein_id_col = "RBP_Accession"
desc_col = "Protein_Type"
source_col = "Bacteriophage"
seq_col = "RBP_Sequence"
output_fasta_file = "unique_phage_proteins1.fasta"
output_excel_file = "phage_processed1.xlsx"

# === Helper Functions ===

def clean_sequence(seq):
    if not isinstance(seq, str):
        return ""
    lines = seq.strip().splitlines()
    lines = [line for line in lines if not line.startswith(">")]
    return "".join(lines).replace(" ", "").upper()

def make_fasta_header(row):
    desc = row.get(desc_col, "")
    source = row.get(source_col, "")
    desc = desc if pd.notna(desc) else ""
    source = source if pd.notna(source) else ""
    return f">{row[protein_id_col]} {desc} [{source}]".strip()

def make_full_fasta(row):
    header = make_fasta_header(row)
    seq = row["CleanSeq"]
    return f"{header}\n{seq}"

def get_taxid_and_accession(phage_name):
    try:
        search = Entrez.esearch(db="nucleotide", term=phage_name, retmode="xml")
        record = Entrez.read(search)
        search.close()
        if not record["IdList"]:
            return None, None
        accession_id = record["IdList"][0]
        handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="gb", retmode="text")
        seq_record = SeqIO.read(handle, "genbank")
        handle.close()
        taxid = None
        for feature in seq_record.features:
            if feature.type == "source":
                for ref in feature.qualifiers.get("db_xref", []):
                    if ref.startswith("taxon:"):
                        taxid = ref.split(":")[1]
                        break
        return taxid, accession_id
    except Exception as e:
        print(f"[{phage_name}] TaxID/Accession fetch error: {e}")
        return None, None

def get_morphotype_from_taxonomy(taxid):
    try:
        handle = Entrez.efetch(db="taxonomy", id=taxid, retmode="xml")
        records = Entrez.read(handle)
        handle.close()
        lineage_ex = records[0].get("LineageEx", [])
        for entry in lineage_ex:
            if entry["Rank"] == "family":
                return entry["ScientificName"]
        return "Unknown morphotype"
    except Exception as e:
        print(f"[{taxid}] Taxonomy error: {e}")
        return "Error"

# === Load data ===
df = pd.read_excel(excel_path, sheet_name=sheet_name)
df = df.dropna(subset=[phage_id_col, protein_id_col, seq_col])

# Step 1: Preserve original sequence
df["OriginalSeq"] = df[seq_col]

# Step 2: Clean sequences
df["CleanSeq"] = df[seq_col].apply(clean_sequence)

# Step 3: Create FASTA format
df["FASTA"] = df.apply(make_full_fasta, axis=1)

# Step 4: Add column with just the sequence part used in FASTA
df["SeqInFASTA"] = df["CleanSeq"]

# Step 5: Map duplicates
seq_to_phageids = {}
for _, row in df.iterrows():
    seq = row["CleanSeq"]
    phageid = row[phage_id_col]
    seq_to_phageids.setdefault(seq, []).append(phageid)

def get_duplicate_phageids(row):
    seq = row["CleanSeq"]
    phageid = row[phage_id_col]
    duplicates = [pid for pid in seq_to_phageids.get(seq, []) if pid != phageid]
    return ", ".join(duplicates) if duplicates else ""

df["DuplicatePhageIDs"] = df.apply(get_duplicate_phageids, axis=1)

# Step 6: Remove duplicate sequences, keep first occurrence
df_unique = df.drop_duplicates(subset=["CleanSeq"]).copy()

# Step 7: Retrieve morphotype and accession
taxid_map = {}
accession_map = {}
morphotype_map = {}

for name in df_unique[phage_id_col].astype(str).unique():
    print(f"Fetching info for: {name}")
    taxid, accession = get_taxid_and_accession(name)
    morphotype = get_morphotype_from_taxonomy(taxid) if taxid else "TaxID not found"
    taxid_map[name] = taxid
    accession_map[name] = accession
    morphotype_map[name] = morphotype
    time.sleep(0.4)

df_unique["TaxID"] = df_unique[phage_id_col].map(taxid_map)
df_unique["AccessionFetched"] = df_unique[phage_id_col].map(accession_map)
df_unique["Morphotype"] = df_unique[phage_id_col].map(morphotype_map)

# Step 8: Write FASTA file
with open(output_fasta_file, "w") as fasta_out:
    for fasta_entry in df_unique["FASTA"]:
        fasta_out.write(fasta_entry + "\n")

# Step 9: Save to Excel
df_unique.to_excel(output_excel_file, index=False)

print(f"FASTA saved to '{output_fasta_file}'")
print(f"Excel saved to '{output_excel_file}'")

Final workflow data processing - NON-VALIDATED:
- fetch the id of bacteriophages from name
- add column with ids
- add fasta format if missing
- create a column with only aa to remove duplicates
- create column with fasta format to create full fasta
- create new column with duplicated identifiers so i dont miss info
- remove duplicates lines based on sequence
- retrieve morphotypes
- create fasta file in fasta format

In [ ]:
import pandas as pd
from Bio import Entrez, SeqIO
import time

# === Configuration ===
Entrez.email = "catiarosario10@gmail.com"
excel_path = "RBP_database_NaoValidadas.csv"
sheet_name = "RBP_database_NaoValidadas"
phage_id_col = "Bacteriophage"
protein_id_col = "RBP accession nr"
desc_col = "Protein type"
source_col = "Bacteriophage"
seq_col = "RBP sequence (aa)"
output_fasta_file = "unique_phage_proteins2.fasta"
output_excel_file = "phage_processed2.xlsx"

# === Helper Functions ===

def clean_sequence(seq):
    if not isinstance(seq, str):
        return ""
    lines = seq.strip().splitlines()
    lines = [line for line in lines if not line.startswith(">")]
    return "".join(lines).replace(" ", "").upper()

def make_fasta_header(row):
    desc = row.get(desc_col, "")
    source = row.get(source_col, "")
    desc = desc if pd.notna(desc) else ""
    source = source if pd.notna(source) else ""
    return f">{row[protein_id_col]} {desc} [{source}]".strip()

def make_full_fasta(row):
    header = make_fasta_header(row)
    seq = row["CleanSeq"]
    return f"{header}\n{seq}"

def get_taxid_and_accession(phage_name):
    try:
        search = Entrez.esearch(db="nucleotide", term=phage_name, retmode="xml")
        record = Entrez.read(search)
        search.close()
        if not record["IdList"]:
            return None, None
        accession_id = record["IdList"][0]
        handle = Entrez.efetch(db="nucleotide", id=accession_id, rettype="gb", retmode="text")
        seq_record = SeqIO.read(handle, "genbank")
        handle.close()
        taxid = None
        for feature in seq_record.features:
            if feature.type == "source":
                for ref in feature.qualifiers.get("db_xref", []):
                    if ref.startswith("taxon:"):
                        taxid = ref.split(":")[1]
                        break
        return taxid, accession_id
    except Exception as e:
        print(f"[{phage_name}] TaxID/Accession fetch error: {e}")
        return None, None

def get_morphotype_from_taxonomy(taxid):
    try:
        handle = Entrez.efetch(db="taxonomy", id=taxid, retmode="xml")
        records = Entrez.read(handle)
        handle.close()
        lineage_ex = records[0].get("LineageEx", [])
        for entry in lineage_ex:
            if entry["Rank"] == "family":
                return entry["ScientificName"]
        return "Unknown morphotype"
    except Exception as e:
        print(f"[{taxid}] Taxonomy error: {e}")
        return "Error"

# === Load data ===
df = pd.read_csv(excel_path)
df = df.dropna(subset=[phage_id_col, protein_id_col, seq_col])

# Step 1: Preserve original sequence
df["OriginalSeq"] = df[seq_col]

# Step 2: Clean sequences
df["CleanSeq"] = df[seq_col].apply(clean_sequence)

# Step 3: Create FASTA format
df["FASTA"] = df.apply(make_full_fasta, axis=1)

# Step 4: Add column with just the sequence part used in FASTA
df["SeqInFASTA"] = df["CleanSeq"]

# Step 5: Map duplicates
seq_to_phageids = {}
for _, row in df.iterrows():
    seq = row["CleanSeq"]
    phageid = row[phage_id_col]
    seq_to_phageids.setdefault(seq, []).append(phageid)

def get_duplicate_phageids(row):
    seq = row["CleanSeq"]
    phageid = row[phage_id_col]
    duplicates = [pid for pid in seq_to_phageids.get(seq, []) if pid != phageid]
    return ", ".join(duplicates) if duplicates else ""

df["DuplicatePhageIDs"] = df.apply(get_duplicate_phageids, axis=1)

# Step 6: Remove duplicate sequences, keep first occurrence
df_unique = df.drop_duplicates(subset=["CleanSeq"]).copy()

# Step 7: Retrieve morphotype and accession
taxid_map = {}
accession_map = {}
morphotype_map = {}

for name in df_unique[phage_id_col].astype(str).unique():
    print(f"Fetching info for: {name}")
    taxid, accession = get_taxid_and_accession(name)
    morphotype = get_morphotype_from_taxonomy(taxid) if taxid else "TaxID not found"
    taxid_map[name] = taxid
    accession_map[name] = accession
    morphotype_map[name] = morphotype
    time.sleep(0.4)

df_unique["TaxID"] = df_unique[phage_id_col].map(taxid_map)
df_unique["AccessionFetched"] = df_unique[phage_id_col].map(accession_map)
df_unique["Morphotype"] = df_unique[phage_id_col].map(morphotype_map)

# Step 8: Write FASTA file
with open(output_fasta_file, "w") as fasta_out:
    for fasta_entry in df_unique["FASTA"]:
        fasta_out.write(fasta_entry + "\n")

# Step 9: Save to Excel
df_unique.to_excel(output_excel_file, index=False)

print(f"FASTA saved to '{output_fasta_file}'")
print(f"Excel saved to '{output_excel_file}'")